In [1]:
import wrangle as w
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
df = w.get_cws_data()

In [3]:
df

,id,total_charges,amount_paid,open,charge_code,description,prop_id,charge_name,sStatus,rent,term,monthly_inc,GuarantorRequired,total_inc,Recommendation,age,risk_score,reason,bad_resident
0,3946,75.00,75.00,0,131,"Uncllctbl key, remote fees",136,"Uncllctbl key, remote fees",Current,1311,15,52500,False,666000,Accept,33,745,Insufficient period of Residence History,1
1,3962,13.66,13.66,0,112,Water 8/1/2021-9/1/2021,136,Utility-water charges,Current,1661,12,65000,False,889992,Accept with Conditions (Extra Deposit),27,758,No Credit Experience,0
2,4050,46.17,46.17,0,155,Damage charges - Clean,136,Damage charges,Current,1412,12,48000,False,576000,Accept with Conditions (Extra Deposit),33,666,Insufficient period of Residence History,1
3,4948,9.76,9.76,0,111,Sewer Charge,140,Utility-sewer charges,Current,1377,18,72000,False,0,A-Criminal History Meets Requirements,39,0,Meets All Property Requirements,0
4,5001,6.85,6.85,0,113,Sewer Base Charge,140,Utility-water flat base charges,Current,2199,13,0,True,54996,Accept with Conditions (Guarantor),23,752,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,74033,1439.37,1439.37,0,74,Rent for 29 days,85,Rent,Current,1489,12,0,True,0,Accept with Conditions (Guarantor),24,708,Insufficient Income to Support Rent,0
4995,74056,150.00,150.00,0,162,Deposit Waiver Fee (Amount subject to screenin...,152,Deposit waiver fee,Current,1859,13,10000,False,121776,Accept,32,786,Meets All Property Requirements,0
4996,74230,5.00,5.00,0,186,Convenience Fee,61,OSP resident convenience fee,Current,1439,12,4350,False,57432,Accept with Conditions (Extra Deposit),36,691,Limited period of Residence History,0
4997,74231,5.00,5.00,0,186,Convenience Fee (ray farmer),83,OSP resident convenience fee,Current,1739,12,7916,False,94992,Accept,47,770,Meets All Property Requirements,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4999 non-null   int64  
 1   total_charges      4999 non-null   float64
 2   amount_paid        4999 non-null   float64
 3   open               4999 non-null   int64  
 4   charge_code        4999 non-null   int64  
 5   description        4999 non-null   object 
 6   prop_id            4999 non-null   int64  
 7   charge_name        4999 non-null   object 
 8   sStatus            4999 non-null   object 
 9   rent               4999 non-null   int64  
 10  term               4999 non-null   int64  
 11  monthly_inc        4999 non-null   int64  
 12  GuarantorRequired  4999 non-null   bool   
 13  total_inc          4999 non-null   int64  
 14  Recommendation     4999 non-null   object 
 15  age                4999 non-null   int64  
 16  risk_score         4999 

In [5]:
y_train, y_val, y_test, train_scaled, val_scaled, test_scaled = w.model_prep(df)

AttributeError: 'NoneType' object has no attribute 'monthly_inc'